In [7]:
 import json
import requests
import string
from collections import Counter


ACCESSTOKEN  = 'EAACEdEose0cBAMmMSI5xJSiCRvwW4fOcr3UKkZCVEJik6V6fVoMohsrnafuO7TtavYhLIQpluuzdBXpeisBSTfpY0ynM5LYv2RmL7oWRtUUA7H0HJx9pUirSoqNz4KspWdGUUQT4Iu6gIfwxPL9uktGgdGy0NUrZAPgvdzogZDZD'
PostUrl ='https://graph.facebook.com/v2.5/15925638948/posts?limit=100&fields=attachments,message,created_time,shares&access_token='+ACCESSTOKEN

res = requests.get(PostUrl)

jd = json.loads(res.text)

PostID= jd['data'][0]['id']
print jd['data'][0]['shares']['count']
len(jd['data'])




23


100

In [8]:

PostUrl ='https://graph.facebook.com/v2.5/15925638948?fields=likes&access_token='+ACCESSTOKEN

res = requests.get(PostUrl)

jd = json.loads(res.text)

Fans= jd['likes']

print Fans


6607306


In [19]:
def insertSQL():
    command="INSERT INTO FBPost (PostNO,Created_Time,Message,ID,AttatchTitle,AttatchMessage,ShareCount,Fans,Crawler_Time,CommentCount,LikeCount) VALUES (?,?,?,?,?,?,?,?,?,?,?)"

    cursor.execute(command,(PostNO,created_time,Postmessage,id,attatchTitle,attatchMessage,ShareCount,Fans,Crawler_Time,CommentCount,LikeCount))
    conn.commit()

# rows = cursor.fetchall()
# for row in rows:

In [20]:
import pyodbc
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=163.14.68.48;DATABASE=FB_NewCrawler;UID=sa;PWD=scucc')
cursor = conn.cursor()

#粉絲
ACCESSTOKEN  = 'EAACEdEose0cBAMmMSI5xJSiCRvwW4fOcr3UKkZCVEJik6V6fVoMohsrnafuO7TtavYhLIQpluuzdBXpeisBSTfpY0ynM5LYv2RmL7oWRtUUA7H0HJx9pUirSoqNz4KspWdGUUQT4Iu6gIfwxPL9uktGgdGy0NUrZAPgvdzogZDZD'
PostUrl ='https://graph.facebook.com/v2.5/15925638948?fields=likes&access_token='+ACCESSTOKEN
res = requests.get(PostUrl)
jd = json.loads(res.text)
Fans= jd['likes']


#PO文
PostUrl ='https://graph.facebook.com/v2.5/15925638948/posts?limit=100&fields=attachments,message,created_time,shares&access_token='+ACCESSTOKEN
res = requests.get(PostUrl)
jd = json.loads(res.text)



PostNO=1
end=0

#時間
Crawler_Time = raw_input("Crawler_Time:")
print Crawler_Time

while( 'next' in jd['paging'] ):
    if end==1:
        break
    
    for post in jd['data']:
        attatchTitle=""
        attatchMessage=""
        CommentCount=0
        LikeCount=0
        ShareCount=0
        Postmessage=""
        
        #例外
        if('message'in post):
            Postmessage=post['message']
        if('shares'in post):
            ShareCount=post['shares']['count']
        created_time=post['created_time']
            
        id=post['id']
        
        #attachments
        if('attachments' in post):
            if('title' in post['attachments']['data'][0]):
                attatchTitle=post['attachments']['data'][0]['title']
            if('description' in post['attachments']['data'][0]):
                attatchMessage=post['attachments']['data'][0]['description']


        CommentCount=Comment(id)
        LikeCount=Like(id)
        
#         print CommentCount
        insertSQL()
        
        if (PostNO%100==0):
            url=jd['paging']['next']

            res=requests.get(url)
            jd = json.loads(res.text)
        PostNO+=1
        
        #停止點
        if (id=='15925638948_10154438254293949'):
            end=1
            print "end"
            break
    
        

cursor.close()
conn.close()   

Crawler_Time:11
11


KeyError: 'paging'

In [14]:
def Comment(PostID):
    CommentUrl ='https://graph.facebook.com/v2.5/'+PostID+'/comments?fields=like_count,message,comment_count,from,id,comments,likes&limit=1000&access_token='+ACCESSTOKEN

    CommentRes = requests.get(CommentUrl)

    CommentJd = json.loads(CommentRes.text)
    
    
    global like_count
    global message
    global comment_count
    global Name
    global UserID
    global CommentID
    global CommentNO
    
    CommentNO=1
    while 'next' in CommentJd['paging'] :
        
        for data in CommentJd['data']:
            like_count= data['like_count']
            message=data['message']
            comment_count=data['comment_count']

            Name= data['from']['name']
            UserID= data['from']['id']
            CommentID=data['id']

            InsertComment()
            if(CommentNO%1000==0):
            
                CommentUrl=CommentJd['paging']['next']
                time.sleep(1)
                CommentRes=requests.get(CommentUrl)
                CommentJd = json.loads(CommentRes.text)
            

            CommentNO+=1
    
          
    for data in CommentJd['data']:
        like_count= data['like_count']
        message=data['message']
        comment_count=data['comment_count']

        Name= data['from']['name']
        UserID= data['from']['id']
        CommentID=data['id']

        InsertComment()

        CommentNO+=1
        
        
    return len(CommentJd['data'])
    
    
    
    
    
    

In [13]:
def InsertComment():
    command="INSERT INTO FBComment (PostNO,CommentNO,Name,Message,CommentID,UserID,LikeCount,ReplyCount) VALUES (?,?,?,?,?,?,?,?)"

    cursor.execute(command,(PostNO,CommentNO,Name,message,CommentID,UserID,like_count,comment_count))
    conn.commit()

In [106]:
Comment('15925638948_10154438254293949')

In [16]:
def InsertLike():
    command="INSERT INTO FBPostLike (PostNO,LikeNO,UserID,UserName) VALUES (?,?,?,?)"

    cursor.execute(command,(PostNO,LikeNO,UserID,UserName))
    conn.commit()

In [37]:

cursor.close()
conn.close() 

In [14]:
import pyodbc
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=163.14.68.48;DATABASE=FB_NewCrawler;UID=sa;PWD=scucc')
cursor = conn.cursor()

In [11]:
def Like(PostID):
    global LikeNO
    global UserName
    global UserID
  
    
    
    LikeNO=1
    LikeUrl ='https://graph.facebook.com/v2.5/'+PostID+'/likes?pretty=0&limit=1000&access_token='+ACCESSTOKEN

    LikeRes = requests.get(LikeUrl)

    LikeJd = json.loads(LikeRes.text)
    while 'next' in LikeJd['paging']:
        for data in LikeJd['data']:
            UserName= data['name']
            UserID= data['id']
            InsertLike()
            if(LikeNO%1000==0):
                LikeUrl=LikeJd['paging']['next']
                #time.sleep(0.1)
                LikeRes=requests.get(LikeUrl)
                LikeJd = json.loads(LikeRes.text)
            LikeNO+=1
            
        
    for data in LikeJd['data']:
        UserName= data['name']
        UserID= data['id']
        InsertLike()
        LikeNO+=1
    return len(LikeJd['data'])
    
    

In [143]:
Like('15925638948_10154438254293949')

760

In [5]:
Test=[4,5,1,2,3]
old=[1,2,3]

for ele in  Test:
    if ele in old:
        index= Test.index(ele)
        theOld=Test[index:]
        theNew=Test[:index]
        break
print theOld
print theNew

[1, 2, 3]
[4, 5]
